# Creating Dimensions and Fact Tables with Pandas

This example demonstrates an ETL (Extract, Transform, Load) process using `pandas` to create dimension tables and a fact table `(star schema)` from raw transactional data. The final outputs are saved as CSV files.

### 1. **Import Libraries**
First, import the necessary libraries:

In [8]:
# Import Libraries
import pandas as pd
from datetime import datetime

### 2. **Load Raw Data**
The raw transactional data is loaded into a pandas DataFrame:

In [12]:
data = {
    "TransactionID": [1, 2, 3, 4],
    "CustomerID": [101, 102, 103, 101],
    "ProductID": [501, 502, 503, 501],
    "Quantity": [2, 1, 3, 1],
    "Price": [10.0, 20.0, 15.0, 10.0],
    "TransactionDate": ["2024-11-01", "2024-11-02", "2024-11-02", "2024-11-03"]
}
df_raw = pd.DataFrame(data)

# Convert TransactionDate to datetime
df_raw["TransactionDate"] = pd.to_datetime(df_raw["TransactionDate"])
print("Raw Data:")
print(df_raw)


Raw Data:
   TransactionID  CustomerID  ProductID  Quantity  Price TransactionDate
0              1         101        501         2   10.0      2024-11-01
1              2         102        502         1   20.0      2024-11-02
2              3         103        503         3   15.0      2024-11-02
3              4         101        501         1   10.0      2024-11-03


### 3. **Create Dimension Tables**
a. Customer Dimension
Extract unique customers and assign each a `CustomerKey`:

In [16]:
customer_dim = df_raw[["CustomerID"]].drop_duplicates().reset_index(drop=True)
customer_dim["CustomerKey"] = customer_dim.index + 1
print("\nCustomer Dimension:")
print(customer_dim)



Customer Dimension:
   CustomerID  CustomerKey
0         101            1
1         102            2
2         103            3



b. Product Dimension
Extract unique products and assign each a `ProductKey`:

In [19]:
product_dim = df_raw[["ProductID"]].drop_duplicates().reset_index(drop=True)
product_dim["ProductKey"] = product_dim.index + 1
print("\nProduct Dimension:")
print(product_dim)



Product Dimension:
   ProductID  ProductKey
0        501           1
1        502           2
2        503           3


c. Time Dimension
Extract unique transaction dates and create date-based attributes:

In [23]:
time_dim = (
    df_raw[["TransactionDate"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
time_dim["TimeKey"] = time_dim.index + 1
time_dim["Year"] = time_dim["TransactionDate"].dt.year
time_dim["Month"] = time_dim["TransactionDate"].dt.month
time_dim["Day"] = time_dim["TransactionDate"].dt.day
print("\nTime Dimension:")
print(time_dim)



Time Dimension:
  TransactionDate  TimeKey  Year  Month  Day
0      2024-11-01        1  2024     11    1
1      2024-11-02        2  2024     11    2
2      2024-11-03        3  2024     11    3


### 4. **Create Fact Table**
Merge the dimension tables with the raw data and calculate total transaction values:

In [28]:
fact_table = df_raw.merge(customer_dim, on="CustomerID")
fact_table = fact_table.merge(product_dim, on="ProductID")
fact_table = fact_table.merge(time_dim, on="TransactionDate")

# Select and rename columns for the fact table
fact_table = fact_table[["TransactionID", "CustomerKey", "ProductKey", "TimeKey", "Quantity", "Price"]]
fact_table["Total"] = fact_table["Quantity"] * fact_table["Price"]
print("\nFact Table:")
print(fact_table)



Fact Table:
   TransactionID  CustomerKey  ProductKey  TimeKey  Quantity  Price  Total
0              1            1           1        1         2   10.0   20.0
1              4            1           1        3         1   10.0   10.0
2              2            2           2        2         1   20.0   20.0
3              3            3           3        2         3   15.0   45.0


### 5. **Save Tables to CSV**
Save the resulting tables to CSV files for future use:

In [31]:
customer_dim.to_csv("customer_dim.csv", index=False)
product_dim.to_csv("product_dim.csv", index=False)
time_dim.to_csv("time_dim.csv", index=False)
fact_table.to_csv("fact_table.csv", index=False)

print("\nData saved to CSV files!")



Data saved to CSV files!


# Summary and Next Steps

### Summary
In this ETL process, we performed the following steps:
1. **Loaded Raw Data**: Imported transactional data into a pandas DataFrame and converted dates to `datetime` format.
2. **Created Dimension Tables**:
   - **Customer Dimension**: Extracted unique customers and assigned a primary key (`CustomerKey`).
   - **Product Dimension**: Extracted unique products and assigned a primary key (`ProductKey`).
   - **Time Dimension**: Extracted unique dates and added attributes such as Year, Month, and Day, along with a primary key (`TimeKey`).
3. **Created the Fact Table**: Merged the dimension tables with the raw data to form a central fact table. Calculated total transaction values (`Quantity * Price`).
4. **Saved Outputs**: Saved all the dimension tables and the fact table as CSV files for further analysis or integration into a data warehouse.

### Next Steps
1. **Enhance Data Quality**:
   - Validate data consistency (e.g., ensure no duplicate or missing values in keys or dimensions).
   - Add data cleansing steps if necessary (e.g., handling outliers or formatting issues).

2. **Data Warehouse Integration**:
   - Load the CSV files into a relational database or data warehouse (e.g., MySQL, PostgreSQL, Snowflake).
   - Establish relationships between dimension tables and the fact table.

3. **Data Analysis**:
   - Perform exploratory data analysis (EDA) on the fact table for insights.
   - Use tools like Power BI, Tableau, or Python for visualization and reporting.

4. **Automation**:
   - Develop a script or pipeline to automate this ETL process for new data batches.
   - Consider using tools like Apache Airflow or cloud services (e.g., AWS Glue, Azure Data Factory).

5. **Advanced Use Cases**:
   - Extend the dimensions (e.g., include product categories or customer demographics).
   - Incorporate additional metrics or KPIs in the fact table (e.g., profit, discount rates).

By following these steps, you can transition this project from a static ETL script to a robust, production-ready data processing pipeline!
